In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import os

from pratplot import colorbar

%matplotlib inline

In [ ]:
base_path = '/Volumes/pss/fits/neurobiases'
exp13_path = os.path.join(base_path, 'exp13.h5')

In [ ]:
results = h5py.File(exp13_path, 'r')

In [ ]:
# Calculate bias on each dataset
a_bias_em = results['a_est'][:] - results['a_true'][:]
b_bias_em = results['b_est'][:] - results['b_true'][:]

In [ ]:
# Aggregate biases across datasets, folds, parameters, and models
# No need for NaN median since this experiment had no selection
a_bias_em_agg = \
    np.nanmedian( # Median across parameters
        np.median( # Median across initializations
            np.mean( # Mean across folds
                np.mean(a_bias_em, axis=3) # Mean across datasets
            , axis=3)
        , axis=2)
    , axis=2)
b_bias_em_agg = \
    np.nanmedian( # NaN Median across parameters
        np.median( # Median across initializations
            np.mean( # Mean across folds
                np.mean(b_bias_em, axis=3) # Mean across datasets
            , axis=3)
        , axis=2)
    , axis=2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

plt.subplots_adjust(wspace=0.75)

img1 = axes[0].imshow(
    a_bias_em_agg,
    vmin=0,
    vmax=0.40,
    cmap='Greys',
    interpolation=None,
    origin='lower')
img2 = axes[1].imshow(
    b_bias_em_agg,
    vmin=-2,
    vmax=2,
    cmap='RdGy',
    interpolation=None,
    origin='upper')

cb1, _ = colorbar.append_colorbar_to_axis(axes[0], img1)
cb2, _ = colorbar.append_colorbar_to_axis(axes[1], img2)

for ax in axes:
    ticks = np.linspace(0, 9, 4, endpoint=True, dtype='int')
    ax.tick_params(labelsize=15)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_yticklabels(np.round(results['tuning_locs'][::3], decimals=1))
    ax.set_xticklabels(np.round(results['coupling_locs'][::3], decimals=1))
    ax.set_ylabel(r'\textbf{Tuning Mean}', fontsize=17)
    ax.set_xlabel(r'\textbf{Coupling Mean}', fontsize=17)
    
cb1.ax.tick_params(labelsize=15)
cb1.set_label(r'\textbf{Coupling Bias}', fontsize=15, rotation=270, labelpad=20)
cb2.ax.tick_params(labelsize=15)
cb2.set_label(r'\textbf{Tuning Bias}', fontsize=15, rotation=270, labelpad=20)
plt.savefig('exp13_bias.pdf', bbox_inches='tight')

In [ ]:
# Aggregate biases across datasets, folds, parameters, but not initializations
a_bias_em_inits = \
    np.nanmedian( # NaN Median across parameters
        np.mean( # Mean across folds
            np.mean(a_bias_em, axis=3) # Mean across datasets
        , axis=3)
    , axis=3)
b_bias_em_inits = \
    np.nanmedian( # NaN Median across parameters
        np.mean( # Mean across folds
            np.mean(b_bias_em, axis=3) # Mean across datasets
        , axis=3)
    , axis=3)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

idxs = [0, 2, 3,  6, 7, 9]
locs = results['tuning_locs'][:]

for idx, loc in enumerate(locs[idxs]):
    axes[0].hist(a_bias_em_inits[5, idxs[idx]],
                 bins=4,
                 color=f'C{idx}',
                 alpha=0.75,
                 label=f'{loc}')
    axes[1].hist(b_bias_em_inits[idxs[idx], 5],
                 bins=4,
                 color=f'C{idx}',
                 alpha=0.75,
                 label=f'$\mu={loc:0.2f}$')


axes[0].set_xlim([0, 0.35])
axes[1].set_xlim([-0.75, 0.75])

axes[1].legend(loc='center left',
               bbox_to_anchor=(1.0, 0.5),
               prop={'size': 13})

for ax in axes:
    ax.tick_params(labelsize=12)
    ax.set_ylabel(r'\textbf{Frequency}', fontsize=15)

axes[0].set_xlabel(r'\textbf{Coupling Bias}', fontsize=15)
axes[1].set_xlabel(r'\textbf{Tuning Bias}', fontsize=15)

plt.tight_layout()
#plt.savefig('exp13_bias_init2.pdf', bbox_inches='tight')

In [ ]:
# Aggregate biases across datasets, folds, parameters, but not initializations
a_var_em_inits = \
    np.median( # NaN Median across parameters
        np.mean( # Mean across folds
            np.var(a_bias_em, axis=3) # Mean across datasets
        , axis=3)
    , axis=3)
b_var_em_inits = \
    np.median( # NaN Median across parameters
        np.mean( # Mean across folds
            np.var(b_bias_em, axis=3) # Mean across datasets
        , axis=3)
    , axis=3)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

idxs = [0, 2, 3,  6, 7, 9]
locs = results['tuning_locs'][:]

for idx, loc in enumerate(locs[idxs]):
    axes[0].hist(a_var_em_inits[5, idxs[idx]],
                 bins=4,
                 color=f'C{idx}',
                 alpha=0.75,
                 label=f'{loc}')
    axes[1].hist(b_var_em_inits[idxs[idx], 5],
                 bins=4,
                 color=f'C{idx}',
                 alpha=0.75,
                 label=f'$\mu={loc:0.2f}$')


axes[0].set_xlim([0, 0.05])
axes[1].set_xlim([0, 0.015])

axes[1].legend(loc='center left',
               bbox_to_anchor=(1.0, 0.5),
               prop={'size': 13})

for ax in axes:
    ax.tick_params(labelsize=12)
    ax.set_ylabel(r'\textbf{Frequency}', fontsize=15)

axes[0].set_xlabel(r'\textbf{Coupling Variance}', fontsize=15)
axes[1].set_xlabel(r'\textbf{Tuning Variance}', fontsize=15)

plt.tight_layout()
plt.savefig('exp13_var_init2.pdf', bbox_inches='tight')